# 种子杯
v3 神经网络

In [42]:
FEATURE1_LEN = 60
FEATURE2_LEN = 25
FEATURE3_LEN = 100
FEATURE4_LEN = 75
EMBEDDING_DIM_CHAR = 256
EMBEDDING_DIM_WORD = 256
CHAR_DIM = 5036
WORD_DIM = 82612

# 0. 初始

In [43]:
import numpy as np
import pandas as pd
from collections import Counter  
from sklearn.metrics import f1_score, accuracy_score, recall_score


In [44]:
train = pd.read_csv("../input/train_a.txt",sep='\t') 
val = pd.read_csv("../input/valid_a.txt",sep='\t') 
test = pd.read_csv("../input/test_a.txt",sep='\t') 

In [45]:
test.head()

,item_id,title_characters,title_words,description_characters,description_words,cate1_id,cate2_id,cate3_id
0,37392888990c22b43962210cdcd54214,"c1208,c165,c94,c321,c15,c462,c557,c266,c207,c2...","w6093,w248,w384,w127,w5315,w2551,w16403,w53,w3...","c201,c434,c373,c27,c442,c303,c29,c2515,c295,c6...","w422,w3248,w1895,w2551,w16403,w83,w100,w13,w38...",NaN,NaN,NaN
1,4d1c0a2a597d0360d12f42a15875613d,"c1675,c609,c677,c687,c290,c735,c15,c7,c1325,c6...","w65765,w352,w11,w4030,w127,w18,w3641,w2223,w12...","c1675,c609,c677,c687,c290,c735,c15,c7,c1325,c6...","w65765,w352,w11,w4030,w127,w18,w1,w2223,w24064...",NaN,NaN,NaN
2,78daa10add22fcdf33a8974bd1a6a2fa,"c539,c1699,c303,c111,c171,c164,c1031,c261,c24,...","w12808,w726,w80,w3421,w105,w682,w6081,w17,w173...","c201,c231,c33,c14,c86,c303,c1085,c1,c478,c132,...","w422,w176,w14,w157,w1421,w1,w205,w141,w196,w11...",NaN,NaN,NaN
3,f6f8abfe9a6c17e25d5820f38def1ab4,"c1208,c165,c217,c33,c15,c7,c266,c207,c291,c611...","w6093,w118,w11,w127,w25420,w31855,w1562,w18,w1...","c611,c611,c471,c299,c1732,c2346,c17,c9,c1,c20,...","w33577,w552,w31855,w18,w1,w61,w13349,w2,w33577...",NaN,NaN,NaN
4,5d5e30daca51c0d66ba41bdd914d7982,"c2525,c314,c471,c299,c125,c264,c4,c4,c1987,c12...","w39351,w2892,w14069,w2577,w6,w4536,w36,w1784,w502","c2012,c19,c209,c125,c264,c4,c4,c3,c24,c97,c1,c...","w6967,w1156,w36787,w2,w4205,w1,w99,w310,w364,w...",NaN,NaN,NaN


# 1 数据预处理

## 1.1 获取y编号 

In [46]:
cate1 = list(train['cate1_id'].unique())
cate2 = list(train['cate2_id'].unique())
cate3 = list(train['cate3_id'].unique())
trainLen = len(train)
valLen = len(val)
testLen = len(test)
# 获取类目树
Tree = {}
for item1 in cate1:
    cate2List = train[train['cate1_id'] == item1]['cate2_id'].unique()
    cate2Dic = {}
    for item2 in cate2List:
        cate3List = train[train['cate2_id'] == item2]['cate3_id'].unique()
        cate2Dic[item2] = list(cate3List)
    Tree[item1] = cate2Dic


## 1.2 构建字典

### 1.2.1 四个属性分析

#### 1.2.1.1 标题字
sort_tc 4001个

60为界限 0.9999075141218822

In [47]:
# 测试 先只考虑标题字
all_word = []
for item in train['title_characters']:
    for word in item.split(','):
        all_word.append(word)
for item in val['title_characters']:
    for word in item.split(','):
        all_word.append(word)
for item in test['title_characters']:
    for word in item.split(','):
        all_word.append(word)
title_characters = dict(Counter(all_word))
sort_tc = sorted(title_characters.items(),key = lambda x:x[1],reverse = True)

del all_word,title_characters

#### 1.2.1.2 标题词
sort_tw 50655个

25界限 0.98084830893129

In [48]:
# 标题词
all_word = []
for item in train['title_words']:
    for word in item.split(','):
        all_word.append(word)
for item in val['title_words']:
    for word in item.split(','):
        all_word.append(word)
for item in test['title_words']:
    for word in item.split(','):
        all_word.append(word)
title_words = dict(Counter(all_word))
sort_tw = sorted(title_words.items(),key = lambda x:x[1],reverse = True)

del all_word,title_words

#### 1.2.1.3 描述字
sort_dc 4805个

100: 0.9035372291230918

In [49]:
# 描述字
all_word = []
for item in train['description_characters']:
    for word in item.split(','):
        all_word.append(word)
for item in val['description_characters']:
    for word in item.split(','):
        all_word.append(word)
for item in test['description_characters']:
    for word in item.split(','):
        all_word.append(word)
description_characters = dict(Counter(all_word))
sort_dc = sorted(description_characters.items(),key = lambda x:x[1],reverse = True)

del all_word,description_characters

#### 1.2.1.4 描述词
sort_dw 64357个

75: 0.9666339408944092,

In [50]:
# 描述词
all_word = []
for item in train['description_words']:
    for word in item.split(','):
        all_word.append(word)
for item in val['description_words']:
    for word in item.split(','):
        all_word.append(word)
for item in test['description_words']:
    for word in item.split(','):
        all_word.append(word)
description_words = dict(Counter(all_word))
sort_dw = sorted(description_words.items(),key = lambda x:x[1],reverse = True)

del all_word,description_words

### 1.2.2 字词分析


In [51]:
# 字 5036
all_word = []
for item in train['title_characters']:
    for word in item.split(','):
        all_word.append(word)
for item in val['title_characters']:
    for word in item.split(','):
        all_word.append(word)
for item in test['title_characters']:
    for word in item.split(','):
        all_word.append(word)
for item in train['description_characters']:
    for word in item.split(','):
        all_word.append(word)
for item in val['description_characters']:
    for word in item.split(','):
        all_word.append(word)
for item in test['description_characters']:
    for word in item.split(','):
        all_word.append(word)
characters = dict(Counter(all_word))
sort_c = sorted(characters.items(),key = lambda x:x[1],reverse = True)

del all_word,characters

In [52]:
# 词 82612
all_word = []
for item in train['title_words']:
    for word in item.split(','):
        all_word.append(word)
for item in val['title_words']:
    for word in item.split(','):
        all_word.append(word)
for item in test['title_words']:
    for word in item.split(','):
        all_word.append(word)
for item in train['description_words']:
    for word in item.split(','):
        all_word.append(word)
for item in val['description_words']:
    for word in item.split(','):
        all_word.append(word)
for item in test['description_words']:
    for word in item.split(','):
        all_word.append(word)
words_ = dict(Counter(all_word))
sort_w = sorted(words_.items(),key = lambda x:x[1],reverse = True)

del all_word,words_

### 1.2.3 构建字典

In [53]:
# 词典 字符-》id编号
word_id_char = {}
for i,item in enumerate(sort_c):
  word_id_char[item[0]]=i+1
  
word_id_word = {}
for i,item in enumerate(sort_w):
  word_id_word[item[0]]=i+1

In [54]:
# # 选择合适长度
# total_len = [len(item.split(',')) for item in train['description_words']]
# total_len_result= dict(Counter(total_len))
# total = 0
# for item in total_len_result:
#   total+=total_len_result[item]
# for item in range(1,120):
#   if item == 1:
#     total_len_result[item] = total_len_result[item]/total
#   else:
#     total_len_result[item] = total_len_result[item]/total + total_len_result[item-1]
# total_len_result

## 1.3 构建X，y

In [55]:
def getX(columnName,featureNum,word_id):
  train_X = np.zeros((trainLen,featureNum))
  for i in range(trainLen):
    words = train[columnName][i].split(',')
    wordsLen = len(words)
    for j in range(min(featureNum,wordsLen)):
      train_X[i][j] = word_id[words[j]]

  val_X = np.zeros((valLen,featureNum))
  for i in range(valLen):
    words = val[columnName][i].split(',')
    wordsLen = len(words)
    for j in range(min(featureNum,wordsLen)):
      val_X[i][j] = word_id[words[j]]
      
  test_X = np.zeros((testLen,featureNum))
  for i in range(testLen):
    words = test[columnName][i].split(',')
    wordsLen = len(words)
    for j in range(min(featureNum,wordsLen)):
      test_X[i][j] = word_id[words[j]]
  print('finish')
  return train_X,val_X,test_X

In [56]:
train_X1,val_X1,test_X1 = getX('title_characters',FEATURE1_LEN,word_id_char)
train_X2,val_X2,test_X2 = getX('title_words',FEATURE2_LEN,word_id_word)
train_X3,val_X3,test_X3 = getX('description_characters',FEATURE3_LEN,word_id_char)
train_X4,val_X4,test_X4 = getX('description_words',FEATURE4_LEN,word_id_word)

finish
finish
finish
finish


In [57]:
# 编号版
train_y1 = [cate1.index(item) for item in train['cate1_id'].tolist()]
train_y2 = [cate2.index(item) for item in train['cate2_id'].tolist()]
train_y3 = [cate3.index(item) for item in train['cate3_id'].tolist()]

val_y1 = [cate1.index(item) for item in val['cate1_id'].tolist()]
val_y2 = [cate2.index(item) for item in val['cate2_id'].tolist()]
val_y3 = [cate3.index(item) for item in val['cate3_id'].tolist()]

In [58]:
# 独热版
from sklearn.preprocessing import OneHotEncoder

ohe1 = OneHotEncoder()
ohe1.fit(np.array(train_y1).reshape(-1,1))

train_y1_hot = ohe1.transform(np.array(train_y1).reshape(-1,1)).toarray()
val_y1_hot = ohe1.transform(np.array(val_y1).reshape(-1,1)).toarray()

ohe2 = OneHotEncoder()
ohe2.fit(np.array(train_y2).reshape(-1,1))

train_y2_hot = ohe2.transform(np.array(train_y2).reshape(-1,1)).toarray()
val_y2_hot = ohe2.transform(np.array(val_y2).reshape(-1,1)).toarray()

ohe3 = OneHotEncoder()
ohe3.fit(np.array(train_y3).reshape(-1,1))

train_y3_hot = ohe3.transform(np.array(train_y3).reshape(-1,1)).toarray()
val_y3_hot = ohe3.transform(np.array(val_y3).reshape(-1,1)).toarray()

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

# 2. 构建网络

## 2.1 辅助函数

In [59]:
import keras
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Dropout,Input,Embedding,LSTM,Bidirectional
from keras.callbacks import Callback,EarlyStopping
from keras.optimizers import Adam,Adamax
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [60]:
# 清空图
def reset_tf_session():
    curr_session = tf.get_default_session()
    # close current session
    if curr_session is not None:
        curr_session.close()
    # reset graph
    K.clear_session()
    # create new session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    s = tf.InteractiveSession(config=config)
    K.set_session(s)
    return s

In [61]:
# 自定义获取val集F1
class GetF1(Callback):
  def on_train_begin(self, logs={}):
    self.f1 = []
    self.f2 = []
    self.f3 = []

  def on_epoch_end(self, epoch, logs={}):
    v1,v2,v3 = measureVal(model)
    self.f1.append(v1)
    self.f2.append(v2)
    self.f3.append(v3)
    return

getF1 = GetF1()

# 保存
modelSave = keras.callbacks.ModelCheckpoint(filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5')

# 早停
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

In [62]:
# 解独热编码
def deOneHot(result,cate=None):
  '''
  如果有cate，则根据cate还原成原分类编号
  '''
  result_list = []
  if cate == None:
    for i,item in enumerate(result):
      result_list.append(list(result[i]).index(max(item)))
  else:
    for i,item in enumerate(result):
      result_list.append(cate[list(result[i]).index(max(item))])    
  return result_list


In [63]:
def measureVal(model):
  '''
  通过val集 预测模型f1效果
  '''
  result1,result2,result3 = model.predict([val_X1,val_X2,val_X3,val_X4])
  result_list1 = deOneHot(result1)
  result_list2 = deOneHot(result2)
  result_list3 = deOneHot(result3)
  v1 = f1_score(result_list1, val_y1, average='macro')
  v2 = f1_score(result_list2, val_y2, average='macro')
  v3 = f1_score(result_list3, val_y3, average='macro')
  print('cate1:',v1)
  print('cate2:',v2)
  print('cate3:',v3)
  print('total:',(v1*0.1+v2*0.3+v3*0.6))
  return v1,v2,v3


def predictTest(model,saveName = None):
  '''
  预测、保存test集结果
  '''
  result1,result2,result3 = model.predict([test_X1,test_X2,test_X3,test_X4])
  result_list1 = deOneHot(result1,cate1)
  result_list2 = deOneHot(result2,cate2)
  result_list3 = deOneHot(result3,cate3)
  test.loc[:,'cate1_id']=result_list1
  test.loc[:,'cate2_id']=result_list2
  test.loc[:,'cate3_id']=result_list3
  if saveName != None:
    test[['item_id','cate1_id','cate2_id','cate3_id']].to_csv(saveName,sep="\t",index=False)

### 2.2 构建网络

In [64]:
 #s =  reset_tf_session()  # 清空之前的图

input_tc = Input(shape=(FEATURE1_LEN,), dtype='int32', name='input_tc')
input_tw = Input(shape=(FEATURE2_LEN,), dtype='int32', name='input_tw')
input_dc = Input(shape=(FEATURE3_LEN,), dtype='int32', name='input_dc')
input_dw = Input(shape=(FEATURE4_LEN,), dtype='int32', name='input_dw')
embedding_char = Embedding(output_dim=EMBEDDING_DIM_CHAR, input_dim=CHAR_DIM)
embedding_word = Embedding(output_dim=EMBEDDING_DIM_WORD, input_dim=WORD_DIM)
conv_1=keras.layers.Conv1D(filters=128,kernel_size=4,strides=2,padding='same',activation='tanh')(embedding_char(input_tc))
maxpool_1=keras.layers.MaxPooling1D(pool_size=4)(conv_1)
flat_1=keras.layers.Flatten()(maxpool_1)
conv_2=keras.layers.Conv1D(filters=128,kernel_size=4,strides=2,padding='same',activation='tanh')(embedding_char(input_dc))
maxpool_2=keras.layers.MaxPooling1D(pool_size=4)(conv_2)
flat_2=keras.layers.Flatten()(maxpool_2)
conv_3=keras.layers.Conv1D(filters=128,kernel_size=4,strides=2,padding='same',activation='tanh')(embedding_word(input_tw))
maxpool_3=keras.layers.MaxPooling1D(pool_size=4)(conv_3)
flat_3=keras.layers.Flatten()(maxpool_3)
conv_4=keras.layers.Conv1D(filters=128,kernel_size=4,strides=2,padding='same',activation='tanh')(embedding_word(input_dw))
maxpool_4=keras.layers.MaxPooling1D(pool_size=4)(conv_4)
flat_4=keras.layers.Flatten()(maxpool_4)
merge_layer = keras.layers.concatenate(([flat_1,flat_2,flat_3,flat_4]))
dense_0 = Dense(1024,activation=None)(merge_layer)
dense_0 = BatchNormalization(momentum=0.9, epsilon=10**-8)(dense_0)
dense_0 = LeakyReLU()(dense_0)
dense_0 = Dense(512,activation=None)(dense_0)
dense_0 = BatchNormalization(momentum=0.9, epsilon=10**-8)(dense_0)
dense_0 = LeakyReLU()(dense_0)
dense_1 = Dense(64,activation=None)(dense_0)
dense_1 = LeakyReLU()(dense_1)
dense_2 = Dense(128,activation=None)(dense_0)
dense_2 = LeakyReLU()(dense_2)
dense_3 = Dense(256,activation=None)(dense_0)
dense_3 = LeakyReLU()(dense_3)

output_1 = Dense(10, activation='softmax', name='output_1')(dense_1)

merge_layer2 = keras.layers.concatenate([output_1,dense_2])
output_2 = Dense(64, activation='softmax', name='output_2')(merge_layer2)

merge_layer3 = keras.layers.concatenate([output_1,output_2,dense_3])
output_3 = Dense(125, activation='softmax', name='output_3')(merge_layer3)

In [65]:
model = Model(inputs=[input_tc,input_tw,input_dc,input_dw], outputs=[output_1,output_2,output_3])

In [66]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_tc (InputLayer)           (None, 60)           0                                            
__________________________________________________________________________________________________
input_dc (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_tw (InputLayer)           (None, 25)           0                                            
__________________________________________________________________________________________________
input_dw (InputLayer)           (None, 75)           0                                            
__________________________________________________________________________________________________
embedding_

In [67]:
model.compile(loss='categorical_crossentropy',
              loss_weights={'output_1': 0.1, 'output_2': 0.3, 'output_3': 0.8},
              optimizer=Adamax(epsilon=10**-8, decay=0.01),
              metrics=['accuracy'])



In [68]:
START_EPOCH = 0
EPOCH = 10

In [69]:
History = model.fit([train_X1,train_X2,train_X3,train_X4], [train_y1_hot,train_y2_hot,train_y3_hot], 
                    batch_size=256,
                    epochs=START_EPOCH+EPOCH,                    
                    shuffle=True, 
                    callbacks=[early_stopping,modelSave,getF1],
                    validation_data=([val_X1,val_X2,val_X3,val_X4], [val_y1_hot,val_y2_hot,val_y3_hot]),
                    initial_epoch=START_EPOCH)

print(History)
# model.save('RNN_2_10.h5')
# #你想要导出的文件的名字
# uploaded = drive.CreateFile({'title': 'RNN_2_10.h5'})
# #改为之前生成文件的名字
# uploaded.SetContentFile('RNN_2_10.h5')
# uploaded.Upload()
# print('Uploaded file with ID {}'.format(uploaded.get('id')))

Train on 140562 samples, validate on 10580 samples
Epoch 1/10
140562/140562 [==============================] - 408s 3ms/step - loss: 1.0346 - output_1_loss: 0.2358 - output_2_loss: 0.6928 - output_3_loss: 1.0040 - output_1_acc: 0.9391 - output_2_acc: 0.8374 - output_3_acc: 0.7505 - val_loss: 0.5964 - val_output_1_loss: 0.1055 - val_output_2_loss: 0.3468 - val_output_3_loss: 0.6023 - val_output_1_acc: 0.9765 - val_output_2_acc: 0.9095 - val_output_3_acc: 0.8336
cate1: 0.9592838455631686
cate2: 0.8624436265435199
cate3: 0.7957229883272229
total: 0.8320952655157066
Epoch 2/10
140562/140562 [==============================] - 400s 3ms/step - loss: 0.3533 - output_1_loss: 0.0705 - output_2_loss: 0.2117 - output_3_loss: 0.3535 - output_1_acc: 0.9847 - output_2_acc: 0.9469 - output_3_acc: 0.9043 - val_loss: 0.5465 - val_output_1_loss: 0.0901 - val_output_2_loss: 0.3053 - val_output_3_loss: 0.5573 - val_output_1_acc: 0.9785 - val_output_2_acc: 0.9166 - val_output_3_acc: 0.8394
cate1: 0.96304012

In [ ]:
History = model.fit([val_X1,val_X2,val_X3,val_X4], [val_y1_hot,val_y2_hot,val_y3_hot], 
                    batch_size=64,
                    epochs=START_EPOCH+EPOCH,                    
                    shuffle=True, 
                    callbacks=[early_stopping,modelSave,getF1],
                    validation_data=([val_X1,val_X2,val_X3,val_X4], [val_y1_hot,val_y2_hot,val_y3_hot]),
#                     verbose=0,
                    initial_epoch=START_EPOCH)

In [ ]:
del model

# 3. 保存

In [ ]:
# !pip install -q pydot
# # 画模型对应的结构图
 from keras.utils import plot_model
 plot_model(model, to_file='model.png')

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
# 预测结果并保存文件
predictTest(model,'result2.txt')

In [ ]:
!ls

In [ ]:
model.save('MyTest.h5')

In [ ]:
# # 加载模型
# from keras.models import load_model 
# model = load_model('model_old.hdf5')